In [7]:
from fastapi import FastAPI
from queue import Queue
import uvicorn
# creating a fast application
app = FastAPI()

# initializing the queue
streamer_queue = Queue()

# fake data streamer
def put_data():
    some_i = 20
    for i in range(20):
      streamer_queue.put(some_i + i)
      
# Creation of thread 
def start_generation():
    thread = Thread(target=put_data)
    time.sleep(0.5)
    thread.start()

# This is an asynchronous function, as it has to wait for
# the queue to be available
async def serve_data():
    # Optinal code to start generation - This can be started anywhere in the code
    start_generation()

    while True:
        # Stopping the retreival process if queue gets empty
        if streamer_queue.empty():
            break
        # Reading and returning the value from the queue
        else:
            value = streamer_queue.get()
            yield str(value)
            streamer_queue.task_done()
        # Providing a buffer timer to the generator, so that we do not
        # break on a false alarm i.e generator is taking time to generate
        # but we are breaking because the queue is empty
        # 2 is an arbitrary number that can be changed based on the choice of
        # the developer
        await asyncio.sleep(2)
        
# Using the endpoint by name /query-stream
@app.get('/query-stream/')
async def stream():
# We use Streaming Response class of Fast API to use the streaming response
    return StreamingResponse(serve_data(), media_type='text/event-stream')




In [14]:
# using requests library to check the response
import requests

# the stream has started in the local host
url = "http://127.0.0.1:7000/query-stream/"

# sending a request and fetching a response which is stored in r
with requests.get(url, stream=True) as r:
    # printing response of each stream
    for chunk in r.iter_content(1024):
        print(chunk)

b'30'
b'31'
b'32'
b'33'
b'34'
b'35'
b'36'
b'37'
b'38'
b'39'
b'40'
b'41'
